# If you are using Colab for free, we highly recommend you activate the T4 GPU
# hardware accelerator. Our models are designed to run with at least 16GB
# of RAM, activating T4 will grant the notebook 16GB of GDDR6 RAM as opposed
# to the ~13GB Colab gives automatically.
# To activate T4:
# 1. click on the "Runtime" tab
# 2. click on "Change runtime type"
# 3. select T4 GPU under Hardware Accelerator
# NOTE: there is a weekly usage limit on using T4 for free


 This example shows an end-to-end recipe for creating a CustomTable, and then creating an Agent process that
    will query the table using natural language.

    Please note that this example is a 'generalized' and updated version of an earlier example -
    "text2sql-end-to-end-2.py" - now using the more powerful CustomTables class integrated into the LLMfx process

    The example shows the following steps:

    1.  Creating a custom table resource from a sample CSV file, included in the slim-sql-tool kit, and also
        available in the Examples section with Structured_Tables (customer_table.csv)

    2  Asking basic natural language questions:
        A.  Looks up the table schema
        B.  Packages the table schema with query
        C.  Runs inference to convert text into SQL
        D.  Queries the database with the generated SQL
        E.  Returns result

    3.  Using CustomtTable class, this can be run on either Postgres or SQLite DB.

    Note: as you substitute for your own CSV and JSON, check out the other examples in this section for loading
    configuration ideas and options.



This notebook is designed for the google collab environment


1) The first cell will allow this notebook to access files in your google drive. The example csv for this notebook is located at https://github.com/llmware-ai/llmware/blob/main/examples/Structured_Tables/customer_table.csv

Upload it to your drive, and ensure that when you input parameters you point to the right path where the csv is located.



In [33]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
!pip install llmware

In [34]:
import os
from llmware.agents import LLMfx
from llmware.resources import CustomTable
from llmware.configs import LLMWareConfig


In [35]:
def build_table(db=None, table_name=None, load_fp=None, load_file=None):
    """Simple example script to take a CSV or JSON/JSONL and create a DB Table."""
    custom_table = CustomTable(db=db, table_name=table_name)
    analysis = custom_table.validate_csv(load_fp, load_file)
    print("update: analysis from validate_csv: ", analysis)

    if load_file.endswith(".csv"):
        output = custom_table.load_csv(load_fp, load_file)
    elif load_file.endswith(".jsonl") or load_file.endswith(".json"):
        output = custom_table.load_json(load_fp, load_file)
    else:
        print("file type not supported for db load")
        return -1

    print("update: output from loading file: ", output)

    sample_range = min(10, len(custom_table.rows))
    for x in range(0, sample_range):
        print("update: sample rows: ", x, custom_table.rows[x])

    updated_schema = custom_table.test_and_remediate_schema(samples=20, auto_remediate=True)
    print("update: updated schema: ", updated_schema)

    custom_table.insert_rows()
    return 1


In [36]:
def agent_natural_language_sql_query(query_list, db=None, table_name=None):
    """Query a CustomTable in natural language."""
    agent = LLMfx()
    agent.load_tool("sql", sample=False, get_logits=True, temperature=0.0)

    for i, query in enumerate(query_list):
        response = agent.query_custom_table(query, db=db, table=table_name)

    for x in range(len(agent.research_list)):
        print("research: ", x, agent.research_list[x])

    return agent.research_list



In [37]:
# Input parameters
db = "sqlite"  # SQLite database path
table_name = "customer_table"

# Path to the CSV file in Google Drive
input_fp = "/content/drive/My Drive"
input_fn = "customer_table.csv"

# Build the table
build_table(db=db, table_name=table_name, load_fp=input_fp, load_file=input_fn)



update: analysis from validate_csv:  {'rows': 20, 'columns': 6, 'conforming_rows_percent': 1.0, 'column_frequency_analysis': {6: 20}, 'nonconforming_rows': []}
update: output from loading file:  {'rows': 19, 'columns': 6, 'schema': {'customer_name': 'text', 'account_number': 'integer', 'customer_level': 'text', 'vip_customer': 'text', 'annual_spend': 'integer', 'user_name': 'text'}, 'skipped_rows': []}
update: sample rows:  0 {'customer_name': 'Martha Williams', 'account_number': '98320893', 'customer_level': 'gold', 'vip_customer': 'yes', 'annual_spend': '63250', 'user_name': 'mwilliams'}
update: sample rows:  1 {'customer_name': 'Susan Soinsin', 'account_number': '53439382', 'customer_level': 'silver', 'vip_customer': 'no', 'annual_spend': '112 ', 'user_name': 'ssinsin'}
update: sample rows:  2 {'customer_name': 'Michael Rogers', 'account_number': '88888444', 'customer_level': 'bronze', 'vip_customer': 'no', 'annual_spend': '3 ', 'user_name': 'rogersm'}
update: sample rows:  3 {'cust

1

In [38]:
query_list = [
    "Which customers have vip customer status of yes?",
    "What is the highest annual spend of any customer?",
    "Which customer has account number 1234953",
    "Which customer has the lowest annual spend?",
    "Is Susan Soinsin a vip customer?"
]

# Execute the queries
agent_natural_language_sql_query(query_list, db=db, table_name=table_name)


update: Launching LLMfx process
step - 	1 - 	creating object - ready to start processing.
step - 	2 - 	loading tool - sql
step - 	3 - 	loading new processing text - 1 new entries
step - 	4 - 	executing function call - deploying - text-to-sql
				 -- query - Which customers have vip customer status of yes?
				 -- table_schema - CREATE TABLE customer_table (customer_name text, account_number integer, customer_level text, vip_customer text, annual_spend integer, user_name text )
step - 	5 - 	executing function call - getting response - sql
				 -- llm_response - SELECT customer_name FROM customer_table WHERE vip_customer = 'yes'
				 -- output type - text
				 -- usage - {'input': 58, 'output': 17, 'total': 75, 'metric': 'tokens', 'processing_time': 1.0267748832702637}
step - 	6 - 	executing research call - executing query on db
				 -- db - sqlite
				 -- sql_query - SELECT customer_name FROM customer_table WHERE vip_customer = 'yes'
step - 	7 - 	executing research  - getting response -

[{'step': 6,
  'tool': 'sql',
  'db_response': [('Martha Williams',),
   ('John Jones',),
   ('Arvind Arora',),
   ('Vinod Aggarwal',),
   ('Allison Winters',),
   ('Olivia Smith',),
   ('Alan Wang',),
   ('Wilmer Rodriguez',),
   ('Eliza Listron',),
   ('Douglas Hudson',),
   ('Martha Williams',),
   ('John Jones',),
   ('Arvind Arora',),
   ('Vinod Aggarwal',),
   ('Allison Winters',),
   ('Olivia Smith',),
   ('Alan Wang',),
   ('Wilmer Rodriguez',),
   ('Eliza Listron',),
   ('Douglas Hudson',)],
  'sql_query': "SELECT customer_name FROM customer_table WHERE vip_customer = 'yes'",
  'query': 'Which customers have vip customer status of yes?',
  'db': 'sqlite',
  'work_item': 'CREATE TABLE customer_table (customer_name text, account_number integer, customer_level text, vip_customer text, annual_spend integer, user_name text )'},
 {'step': 11,
  'tool': 'sql',
  'db_response': [(93540,)],
  'sql_query': 'SELECT MAX(annual_spend) FROM customer_table',
  'query': 'What is the highest a